In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

from sklearn.metrics import make_scorer, recall_score, precision_score, confusion_matrix, classification_report
from sklearn.preprocessing import RobustScaler, OneHotEncoder, PowerTransformer, PolynomialFeatures
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.utils import resample

from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV, RandomizedSearchCV
import imblearn
from imblearn.over_sampling import SMOTE
pd.set_option('display.max_columns', None)

import joblib

https://www.kaggle.com/volodymyrgavrysh/bank-marketing-campaigns-dataset

__Conducted campaigns were based mostly on direct phone calls, offering bank client to place a term deposit.
If after all marking afforts client had agreed to place deposit - target variable marked 'yes', otherwise 'no'__

# Feature Desc

### bank client data:

1. age
    - (numeric)

2. job : type of job 
    - (categorical: "admin.","blue-collar","entrepreneur","housemaid","management","retired","self-employed","services","student","technician","unemployed","unknown")

3. marital : marital status 
    - (categorical: "divorced","married","single","unknown"; note: "divorced" means divorced or widowed)

4. education 
    - (categorical: "basic.4y","basic.6y","basic.9y","high.school","illiterate","professional.course","university.degree","unknown")

5. default: has credit in default? (lalai bayar (?))
    - (categorical: "no","yes","unknown")

6. housing: has housing loan? 
    - (categorical: "no","yes","unknown")

7. loan: has personal loan?
    - (categorical: "no","yes","unknown")
    
### related with the last contact of the current campaign:
8. contact: contact communication type
    - (categorical: "cellular","telephone")
9. month: last contact month of year
    - (categorical: "jan", "feb", "mar", …, "nov", "dec")

10. dayofweek: last contact day of the week
    - (categorical: "mon","tue","wed","thu","fri")

11. duration: last contact duration, in seconds (numeric). ==> (obvious feature (?))
    - __Important note__: this attribute highly affects the output target (e.g., if duration=0 then y="no"). 
    - Yet, the duration is not known before a call is performed. Also, after the end of the call y is obviously known. Thus, this input should only be included for benchmark purposes and should be discarded if the intention is to have a realistic predictive model.

### other attributes:
12. campaign: number of contacts performed during this campaign and for this client 
    - (numeric, includes last contact)

13. pdays: number of days that passed by after the client was last contacted from a previous campaign
    - (numeric; 999 means client was not previously contacted)

14. previous: number of contacts performed before this campaign and for this client
    - (numeric)

15. poutcome: outcome of the previous marketing campaign 
    - (categorical: "failure","nonexistent","success")

### social and economic context attributes
16. emp.var.rate: employment variation rate - quarterly indicator (???)
    - (numeric)

17. cons.price.idx: consumer price index - monthly indicator
    - changes in the price level of a weighted average __market basket__ of consumer goods and services purchased by households
    - affect inflation
    - (numeric)

18. cons.conf.idx: consumer confidence index - monthly indicator
    - degree of __consumers optimism__ are expressing through their activities of savings and spending.
    - affect consumer behavior
    - (numeric)

19. euribor3m: euribor 3 month rate - daily indicator
    - Euribor (euro interbank offered rate)
    - ??
    - (numeric)

20. nr.employed: number of employees - quarterly indicator
    - Number of employed persons for a quarter. (for the bank ??)
    - (numeric)

### Output variable (desired target):

21. y - has the client subscribed a term deposit?
    - (binary: "yes","no")

In [2]:
df = pd.read_csv('bank-additional-full.csv', sep=';')
df.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,duration,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,261,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,149,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,226,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,151,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,307,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


In [3]:
df.education = df.education.apply(lambda x : x.replace(".", " "))

# Data Description

In [4]:
dfDesc = []
for i in df.columns:
    if df[i].nunique() > 3 :
        bla = df[i].drop_duplicates().sample(3).values
    else :
        bla = df[i].drop_duplicates().values
    
    dfDesc.append([
        i,
        df[i].dtypes,
        df[i].isna().sum(),
        round((((df[i].isna().sum()) / len(df)) * 100), 2),
        df[i].nunique(),
        bla
    ])
    
pd.DataFrame(dfDesc, columns=[
    "Data Features",
    "Data Types",
    "Null",
    "Null Percentage",
    "Unique",
    "Unique Sample"
])

,Data Features,Data Types,Null,Null Percentage,Unique,Unique Sample
0,age,int64,0,0.0,78,"[38, 55, 72]"
1,job,object,0,0.0,12,"[technician, blue-collar, unemployed]"
2,marital,object,0,0.0,4,"[divorced, single, unknown]"
3,education,object,0,0.0,8,"[illiterate, unknown, professional course]"
4,default,object,0,0.0,3,"[no, unknown, yes]"
5,housing,object,0,0.0,3,"[no, yes, unknown]"
6,loan,object,0,0.0,3,"[no, yes, unknown]"
7,contact,object,0,0.0,2,"[telephone, cellular]"
8,month,object,0,0.0,10,"[jun, oct, apr]"
9,day_of_week,object,0,0.0,5,"[tue, mon, fri]"


__Insisght__:
- Tidak ada data null

# Stats Description

In [5]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
age,41188.0,40.024060,10.421250,17.000,32.000,38.000,47.000,98.000
duration,41188.0,258.285010,259.279249,0.000,102.000,180.000,319.000,4918.000
campaign,41188.0,2.567593,2.770014,1.000,1.000,2.000,3.000,56.000
pdays,41188.0,962.475454,186.910907,0.000,999.000,999.000,999.000,999.000
previous,41188.0,0.172963,0.494901,0.000,0.000,0.000,0.000,7.000
emp.var.rate,41188.0,0.081886,1.570960,-3.400,-1.800,1.100,1.400,1.400
cons.price.idx,41188.0,93.575664,0.578840,92.201,93.075,93.749,93.994,94.767
cons.conf.idx,41188.0,-40.502600,4.628198,-50.800,-42.700,-41.800,-36.400,-26.900
euribor3m,41188.0,3.621291,1.734447,0.634,1.344,4.857,4.961,5.045
nr.employed,41188.0,5167.035911,72.251528,4963.600,5099.100,5191.000,5228.100,5228.100


In [6]:
df.describe(include='O')

,job,marital,education,default,housing,loan,contact,month,day_of_week,poutcome,y
count,41188,41188,41188,41188,41188,41188,41188,41188,41188,41188,41188
unique,12,4,8,3,3,3,2,10,5,3,2
top,admin.,married,university degree,no,yes,no,cellular,may,thu,nonexistent,no
freq,10422,24928,12168,32588,21576,33950,26144,13769,8623,35563,36548


# Cek Imbalance Data dan Pembagian data numerik dan kategorik

In [7]:
df['y'].value_counts()

no     36548
yes     4640
Name: y, dtype: int64

In [8]:
num=['age', 'duration', 'campaign', 'pdays', 'previous', 'emp.var.rate', 'cons.price.idx', 'cons.conf.idx', 'euribor3m', 'nr.employed']
cat=[i for i in df.columns if i not in num]

# Cek Outliers

In [9]:
Desc_outliers = []

for i in df.describe().columns:
    Q1 = df.describe()[i]['25%']
    Q3 = df.describe()[i]['75%']
    IQR = Q3 - Q1
    upper_fence = Q3 + (1.5*IQR)
    lower_fence = Q1 - (1.5*IQR)
    hitung = 0
    
    for j in df[i]:
        if (j < lower_fence) | (j > upper_fence) :
            hitung+=1
    Desc_outliers.append([
        i,
        hitung,
        upper_fence,
        lower_fence
    ])

out = pd.DataFrame(Desc_outliers, columns=[
    "Data Features",
    "Jumlah Outliers",
    "Upper Fence",
    "Lower Fence"
])
out

,Data Features,Jumlah Outliers,Upper Fence,Lower Fence
0,age,469,69.5000,9.5000
1,duration,2963,644.5000,-223.5000
2,campaign,2406,6.0000,-2.0000
3,pdays,1515,999.0000,999.0000
4,previous,5625,0.0000,0.0000
5,emp.var.rate,0,6.2000,-6.6000
6,cons.price.idx,0,95.3725,91.6965
7,cons.conf.idx,447,-26.9500,-52.1500
8,euribor3m,0,10.3865,-4.0815
9,nr.employed,0,5421.6000,4905.6000


__Insight__:

- Pd kolom age, duration, campaign, pdays, previous dan cons.conf.idx memiliki outliers yg cukup banyak

In [10]:
df['previous'].value_counts()

0    35563
1     4561
2      754
3      216
4       70
5       18
6        5
7        1
Name: previous, dtype: int64

__Insight__:

- Pd kolom pdays bentuk boxplotnya unik karena nilai 999 nya sangat lebih banyak dari nilai lainnya memiliki rentang yang jauh dengan 999
- Pd kolom previous bentuk boxplotnya unik karena nilai 0 nya sangat lebih banyak dari niali lainnya. antar nilai memiliki rentang 1. 

# ML

### GOALS 

__Nurunin jumlah False Positif__ 

__Naikin Nilai Precision kelas 1 (YES)__

### PR

1. coba label encoder untuk categorical

2. coba di bin age, duration(per 180/3 meni, yng diatas 1000 jadi 1000++)

## Binning Data

Lakukan binning pada kolom age dan duration

In [11]:
age_bin = [-1, 10, 20, 40, df['age'].max()] ## Membuat Kelompok Usia
labels = ['Kids', 'Teenager', 'Adult', 'Elder'] ## Label untuk Kelompok Usia
df['age_bin'] = pd.cut(df['age'], bins=age_bin, labels=labels)

In [12]:
duration_bin = [-1, 180, 360, 540, 720, 900,df['duration'].max()]
label_duration = ['0-180', '180-360', '360-540', '540-720', '720-900', '>900']
df['duration_bin'] = pd.cut(df['duration'], bins=duration_bin, labels=label_duration)

In [13]:
df['y'].value_counts()

no     36548
yes     4640
Name: y, dtype: int64

1 ==> Nasabah subscribe deposit (yes)

0 ==> Nasabah TIDAK subscribe deposit (no)

In [14]:
df['y'] = df['y'].map({'yes' : 1, 'no' : 0})

In [15]:
df['y'].dtypes

dtype('int64')

## Cek Imbalance Data

In [16]:
round(df['y'].value_counts()/len(df)*100, 2)

0    88.73
1    11.27
Name: y, dtype: float64

In [17]:
df = df.drop(columns=['age', 'duration'])

## Splitting Data

In [18]:
X = df.drop(columns='y')
y = df['y']

In [19]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=.20, random_state=42)

## Pipeline

In [20]:
df.columns

Index(['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact',
       'month', 'day_of_week', 'campaign', 'pdays', 'previous', 'poutcome',
       'emp.var.rate', 'cons.price.idx', 'cons.conf.idx', 'euribor3m',
       'nr.employed', 'y', 'age_bin', 'duration_bin'],
      dtype='object')

In [90]:
num_columns = ['campaign', 'pdays', 'previous', 'emp.var.rate', 'cons.price.idx', 'cons.conf.idx', 'euribor3m', 'nr.employed']
cat_columns = ['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'month', 'day_of_week', 'poutcome', 'age_bin', 'duration_bin']

In [22]:
num_columns

['campaign',
 'pdays',
 'previous',
 'emp.var.rate',
 'cons.price.idx',
 'cons.conf.idx',
 'euribor3m',
 'nr.employed']

In [23]:
cat_columns

['job',
 'marital',
 'education',
 'default',
 'housing',
 'loan',
 'contact',
 'month',
 'day_of_week',
 'poutcome',
 'y',
 'age_bin',
 'duration_bin']

In [91]:
numeric_pipeline = Pipeline([
    ('scaler', RobustScaler())
#     ('poly', PolynomialFeatures(degree=2, include_bias=False)),
#     ('power', PowerTransformer(method='yeo-johnson'))
])

categoric_pipeline = Pipeline([
    ('encoder', OneHotEncoder())
])

preprocessor = ColumnTransformer([
    ('numeric', numeric_pipeline, num_columns),
    ('categorical', categoric_pipeline, cat_columns)
])

pipe_DT = Pipeline([
    ('prep', preprocessor),
    ('algo', DecisionTreeClassifier())
])

pipe_KNN = Pipeline([
    ('prep', preprocessor),
    ('algo', KNeighborsClassifier())
])

## Def Eva Metrix

Fungsi def yg isinya evaluation metrix.
Evaluation matrix yang dipakai kali ini adalah Cunfusion matrix dan Classification report

In [27]:
def conf_mat (Model, X_train, X_test, y_train, y_test,Nama):
    y_pred_test = Model.predict(X_test)
    cm_test = confusion_matrix(y_test, y_pred_test, labels=[1,0])
    df = pd.DataFrame(cm_test, index = ['Akt1', 'Akt0'], columns=['Pred1', 'Pred0'])
    print( 'Classification report data TEST ' + Nama + '\n\n', classification_report(y_test, y_pred_test))
    
    y_pred_train = Model.predict(X_train)
    print( 'Classification report data TRAIN ' + Nama + '\n\n', classification_report(y_train, y_pred_train))
#     return plt.title('Confusion matrix data test ' + Nama + '\n'), sns.heatmap(df, annot=True)
    print ('Confusion matrix data test ' + Nama + '\n')
    return df

In [28]:
def prec_rec (Model, X_test, y_test, Nama):
    data = {}
    prec = []
    rec = []
    # for i in Model :
    y_pred_ts = Model.predict(X_test)
    precision = precision_score(y_test, y_pred_ts)
    recall = recall_score(y_test, y_pred_ts)
    prec.append(precision)
    rec.append(recall)
    data[Nama] = [prec[0], rec[0]]
    
    df = pd.DataFrame(data, index=['Precison', 'Recall'])
    return df

## Over Sampling

Melakukan teknik Oversampling untuk mengatasa data imbalance sebelum modelling

In [29]:
X_train

,job,marital,education,default,housing,loan,contact,month,day_of_week,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,age_bin,duration_bin
25611,blue-collar,married,basic 9y,unknown,no,no,cellular,nov,wed,4,999,0,nonexistent,-0.1,93.200,-42.0,4.120,5195.8,Elder,180-360
26010,entrepreneur,married,university degree,no,no,no,telephone,nov,wed,2,999,1,failure,-0.1,93.200,-42.0,4.120,5195.8,Adult,180-360
40194,retired,married,basic 4y,no,no,no,cellular,jul,mon,1,999,0,nonexistent,-1.7,94.215,-40.3,0.870,4991.6,Elder,>900
297,admin.,married,university degree,no,yes,no,telephone,may,mon,2,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,Adult,0-180
36344,retired,divorced,university degree,no,no,no,cellular,jun,tue,2,999,0,nonexistent,-2.9,92.963,-40.8,1.262,5076.2,Elder,360-540
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40205,services,single,high school,no,yes,no,cellular,jul,tue,1,999,0,nonexistent,-1.7,94.215,-40.3,0.876,4991.6,Adult,180-360
27316,technician,married,professional course,no,yes,no,cellular,nov,fri,1,999,1,failure,-0.1,93.200,-42.0,4.021,5195.8,Elder,0-180
14392,admin.,married,basic 9y,no,no,yes,cellular,jul,mon,4,999,0,nonexistent,1.4,93.918,-42.7,4.962,5228.1,Elder,0-180
7494,admin.,married,university degree,no,no,no,telephone,may,fri,1,999,0,nonexistent,1.1,93.994,-36.4,4.864,5191.0,Adult,0-180


In [30]:
y_train

25611    0
26010    0
40194    1
297      0
36344    0
        ..
40205    0
27316    0
14392    0
7494     0
10747    0
Name: y, Length: 32950, dtype: int64

In [31]:
df_train = pd.concat([X_train,y_train], axis=1)

In [32]:
df_train.head()

,job,marital,education,default,housing,loan,contact,month,day_of_week,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,age_bin,duration_bin,y
25611,blue-collar,married,basic 9y,unknown,no,no,cellular,nov,wed,4,999,0,nonexistent,-0.1,93.200,-42.0,4.120,5195.8,Elder,180-360,0
26010,entrepreneur,married,university degree,no,no,no,telephone,nov,wed,2,999,1,failure,-0.1,93.200,-42.0,4.120,5195.8,Adult,180-360,0
40194,retired,married,basic 4y,no,no,no,cellular,jul,mon,1,999,0,nonexistent,-1.7,94.215,-40.3,0.870,4991.6,Elder,>900,1
297,admin.,married,university degree,no,yes,no,telephone,may,mon,2,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,Adult,0-180,0
36344,retired,divorced,university degree,no,no,no,cellular,jun,tue,2,999,0,nonexistent,-2.9,92.963,-40.8,1.262,5076.2,Elder,360-540,0


In [33]:
df_train['y'].value_counts()

0    29238
1     3712
Name: y, dtype: int64

In [34]:
non_default = df_train[df_train['y'] == 0] # kelas majority

In [35]:
default = df_train[df_train['y'] == 1] # kelas minority

In [36]:
default

,job,marital,education,default,housing,loan,contact,month,day_of_week,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,age_bin,duration_bin,y
40194,retired,married,basic 4y,no,no,no,cellular,jul,mon,1,999,0,nonexistent,-1.7,94.215,-40.3,0.870,4991.6,Elder,>900,1
37395,student,single,basic 9y,no,no,no,telephone,aug,wed,1,999,0,nonexistent,-2.9,92.201,-31.4,0.879,5076.2,Adult,360-540,1
31118,blue-collar,married,basic 9y,no,yes,no,cellular,may,wed,1,999,0,nonexistent,-1.8,92.893,-46.2,1.334,5099.1,Adult,>900,1
39226,admin.,single,university degree,no,no,yes,cellular,mar,tue,2,3,1,success,-1.8,93.369,-34.8,0.652,5008.7,Adult,180-360,1
39224,unemployed,divorced,university degree,no,no,no,cellular,mar,tue,3,6,3,success,-1.8,93.369,-34.8,0.652,5008.7,Elder,180-360,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37603,retired,married,professional course,no,yes,yes,cellular,aug,mon,1,999,0,nonexistent,-2.9,92.201,-31.4,0.861,5076.2,Elder,180-360,1
36595,student,single,high school,no,no,no,cellular,jun,thu,1,999,0,nonexistent,-2.9,92.963,-40.8,1.260,5076.2,Adult,360-540,1
8135,technician,single,professional course,unknown,no,no,telephone,jun,mon,4,999,0,nonexistent,1.4,94.465,-41.8,4.865,5228.1,Adult,360-540,1
35126,technician,single,university degree,no,yes,yes,cellular,may,fri,3,999,0,nonexistent,-1.8,92.893,-46.2,1.250,5099.1,Adult,360-540,1


In [37]:
default_oversample = resample(default, 
                           replace = True, 
                           n_samples = len(non_default),
                           random_state=42)

In [38]:
df_OverSample= pd.concat([non_default, default_oversample])

In [39]:
df_OverSample['y'].value_counts()

0    29238
1    29238
Name: y, dtype: int64

In [40]:
df_OverSample

,job,marital,education,default,housing,loan,contact,month,day_of_week,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,age_bin,duration_bin,y
25611,blue-collar,married,basic 9y,unknown,no,no,cellular,nov,wed,4,999,0,nonexistent,-0.1,93.200,-42.0,4.120,5195.8,Elder,180-360,0
26010,entrepreneur,married,university degree,no,no,no,telephone,nov,wed,2,999,1,failure,-0.1,93.200,-42.0,4.120,5195.8,Adult,180-360,0
297,admin.,married,university degree,no,yes,no,telephone,may,mon,2,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,Adult,0-180,0
36344,retired,divorced,university degree,no,no,no,cellular,jun,tue,2,999,0,nonexistent,-2.9,92.963,-40.8,1.262,5076.2,Elder,360-540,0
21806,admin.,single,university degree,no,no,no,cellular,aug,wed,2,999,0,nonexistent,1.4,93.444,-36.1,4.964,5228.1,Adult,180-360,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12425,entrepreneur,married,basic 6y,no,yes,yes,cellular,jul,mon,1,999,0,nonexistent,1.4,93.918,-42.7,4.960,5228.1,Adult,720-900,1
38689,admin.,single,high school,no,yes,no,cellular,nov,mon,1,3,1,success,-3.4,92.649,-30.1,0.715,5017.5,Adult,180-360,1
34610,blue-collar,single,basic 9y,no,yes,yes,cellular,may,thu,4,999,0,nonexistent,-1.8,92.893,-46.2,1.266,5099.1,Adult,180-360,1
37465,admin.,single,university degree,no,yes,yes,cellular,aug,thu,3,3,1,success,-2.9,92.201,-31.4,0.873,5076.2,Adult,180-360,1


In [41]:
X_train_OS = df_OverSample.drop(columns='y')
y_train_OS = df_OverSample['y']

## Modelling

1 ==> Nasabah subscribe deposit (yes)

0 ==> Nasabah TIDAK subscribe deposit (no)

__DT__

In [42]:
df.columns

Index(['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact',
       'month', 'day_of_week', 'campaign', 'pdays', 'previous', 'poutcome',
       'emp.var.rate', 'cons.price.idx', 'cons.conf.idx', 'euribor3m',
       'nr.employed', 'y', 'age_bin', 'duration_bin'],
      dtype='object')

In [68]:
num_columns = ['campaign', 'pdays', 'previous', 'emp.var.rate', 'cons.price.idx', 'cons.conf.idx', 'euribor3m', 'nr.employed']
cat_columns = ['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'month', 'day_of_week', 'poutcome', 'age_bin', 'duration_bin']

In [69]:
df[num_columns]

,campaign,pdays,previous,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed
0,1,999,0,1.1,93.994,-36.4,4.857,5191.0
1,1,999,0,1.1,93.994,-36.4,4.857,5191.0
2,1,999,0,1.1,93.994,-36.4,4.857,5191.0
3,1,999,0,1.1,93.994,-36.4,4.857,5191.0
4,1,999,0,1.1,93.994,-36.4,4.857,5191.0
...,...,...,...,...,...,...,...,...
41183,1,999,0,-1.1,94.767,-50.8,1.028,4963.6
41184,1,999,0,-1.1,94.767,-50.8,1.028,4963.6
41185,2,999,0,-1.1,94.767,-50.8,1.028,4963.6
41186,1,999,0,-1.1,94.767,-50.8,1.028,4963.6


In [70]:
df[cat_columns]

,job,marital,education,default,housing,loan,contact,month,day_of_week,poutcome,age_bin,duration_bin
0,housemaid,married,basic 4y,no,no,no,telephone,may,mon,nonexistent,Elder,180-360
1,services,married,high school,unknown,no,no,telephone,may,mon,nonexistent,Elder,0-180
2,services,married,high school,no,yes,no,telephone,may,mon,nonexistent,Adult,180-360
3,admin.,married,basic 6y,no,no,no,telephone,may,mon,nonexistent,Adult,0-180
4,services,married,high school,no,no,yes,telephone,may,mon,nonexistent,Elder,180-360
...,...,...,...,...,...,...,...,...,...,...,...,...
41183,retired,married,professional course,no,yes,no,cellular,nov,fri,nonexistent,Elder,180-360
41184,blue-collar,married,professional course,no,no,no,cellular,nov,fri,nonexistent,Elder,360-540
41185,retired,married,university degree,no,yes,no,cellular,nov,fri,nonexistent,Elder,180-360
41186,technician,married,professional course,no,no,no,cellular,nov,fri,nonexistent,Elder,360-540


In [71]:
num_columns

['campaign',
 'pdays',
 'previous',
 'emp.var.rate',
 'cons.price.idx',
 'cons.conf.idx',
 'euribor3m',
 'nr.employed']

In [72]:
df.corr()

,campaign,pdays,previous,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
campaign,1.000000,0.052584,-0.079141,0.150754,0.127836,-0.013733,0.135133,0.144095,-0.066357
pdays,0.052584,1.000000,-0.587514,0.271004,0.078889,-0.091342,0.296899,0.372605,-0.324914
previous,-0.079141,-0.587514,1.000000,-0.420489,-0.203130,-0.050936,-0.454494,-0.501333,0.230181
emp.var.rate,0.150754,0.271004,-0.420489,1.000000,0.775334,0.196041,0.972245,0.906970,-0.298334
cons.price.idx,0.127836,0.078889,-0.203130,0.775334,1.000000,0.058986,0.688230,0.522034,-0.136211
cons.conf.idx,-0.013733,-0.091342,-0.050936,0.196041,0.058986,1.000000,0.277686,0.100513,0.054878
euribor3m,0.135133,0.296899,-0.454494,0.972245,0.688230,0.277686,1.000000,0.945154,-0.307771
nr.employed,0.144095,0.372605,-0.501333,0.906970,0.522034,0.100513,0.945154,1.000000,-0.354678
y,-0.066357,-0.324914,0.230181,-0.298334,-0.136211,0.054878,-0.307771,-0.354678,1.000000


In [73]:
X_train_OS

,job,marital,education,default,housing,loan,contact,month,day_of_week,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,age_bin,duration_bin
25611,blue-collar,married,basic 9y,unknown,no,no,cellular,nov,wed,4,999,0,nonexistent,-0.1,93.200,-42.0,4.120,5195.8,Elder,180-360
26010,entrepreneur,married,university degree,no,no,no,telephone,nov,wed,2,999,1,failure,-0.1,93.200,-42.0,4.120,5195.8,Adult,180-360
297,admin.,married,university degree,no,yes,no,telephone,may,mon,2,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,Adult,0-180
36344,retired,divorced,university degree,no,no,no,cellular,jun,tue,2,999,0,nonexistent,-2.9,92.963,-40.8,1.262,5076.2,Elder,360-540
21806,admin.,single,university degree,no,no,no,cellular,aug,wed,2,999,0,nonexistent,1.4,93.444,-36.1,4.964,5228.1,Adult,180-360
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12425,entrepreneur,married,basic 6y,no,yes,yes,cellular,jul,mon,1,999,0,nonexistent,1.4,93.918,-42.7,4.960,5228.1,Adult,720-900
38689,admin.,single,high school,no,yes,no,cellular,nov,mon,1,3,1,success,-3.4,92.649,-30.1,0.715,5017.5,Adult,180-360
34610,blue-collar,single,basic 9y,no,yes,yes,cellular,may,thu,4,999,0,nonexistent,-1.8,92.893,-46.2,1.266,5099.1,Adult,180-360
37465,admin.,single,university degree,no,yes,yes,cellular,aug,thu,3,3,1,success,-2.9,92.201,-31.4,0.873,5076.2,Adult,180-360


In [74]:
xlabel = X_train_OS.copy(deep=True)

In [75]:
xlabel.columns

Index(['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact',
       'month', 'day_of_week', 'campaign', 'pdays', 'previous', 'poutcome',
       'emp.var.rate', 'cons.price.idx', 'cons.conf.idx', 'euribor3m',
       'nr.employed', 'age_bin', 'duration_bin'],
      dtype='object')

In [76]:
xlabel.education.unique()

array(['basic 9y', 'university degree', 'basic 4y', 'high school',
       'professional course', 'unknown', 'basic 6y', 'illiterate'],
      dtype=object)

In [77]:
X_test_label = X_test.copy(deep=True)

In [78]:
xlabel['education'] = xlabel['education'].map(
    {
        "unknown" : 0,
        "illiterate" : 0,
        "basic 4y" : 1,
        "basic 6y" : 2,
        "basic 9y" : 3,
        "high school" : 4,
        "university degree" : 5,
        "professional course" : 6
    }
)

In [79]:
X_test_label['education'] = X_test_label['education'].map(
    {
        "unknown" : 0,
        "illiterate" : 0,
        "basic 4y" : 1,
        "basic 6y" : 2,
        "basic 9y" : 3,
        "high school" : 4,
        "university degree" : 5,
        "professional course" : 6
    }
)

In [80]:
xlabel['age_bin'].unique()

['Elder', 'Adult', 'Teenager']
Categories (3, object): ['Teenager' < 'Adult' < 'Elder']

In [81]:
xlabel['age_bin'] = xlabel['age_bin'].map(
    {
        "Teenager" : 0,
        "Adult" : 1,
        "Elder" : 2
    }
)

In [82]:
X_test_label['age_bin'] = X_test_label['age_bin'].map(
    {
        "Teenager" : 0,
        "Adult" : 1,
        "Elder" : 2
    }
)

In [83]:
xlabel['duration_bin'].unique()

['180-360', '0-180', '360-540', '540-720', '>900', '720-900']
Categories (6, object): ['0-180' < '180-360' < '360-540' < '540-720' < '720-900' < '>900']

In [84]:
xlabel['duration_bin'] = xlabel['duration_bin'].map(
    {
        '180-360' : 1, '0-180' : 0, '360-540' : 2, '540-720' : 3, '>900' : 5, '720-900' : 4
    }
)

In [85]:
X_test_label['duration_bin'] = X_test_label['duration_bin'].map(
    {
        '180-360' : 1, '0-180' : 0, '360-540' : 2, '540-720' : 3, '>900' : 5, '720-900' : 4
    }
)

In [86]:
X_train_OS

,job,marital,education,default,housing,loan,contact,month,day_of_week,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,age_bin,duration_bin
25611,blue-collar,married,basic 9y,unknown,no,no,cellular,nov,wed,4,999,0,nonexistent,-0.1,93.200,-42.0,4.120,5195.8,Elder,180-360
26010,entrepreneur,married,university degree,no,no,no,telephone,nov,wed,2,999,1,failure,-0.1,93.200,-42.0,4.120,5195.8,Adult,180-360
297,admin.,married,university degree,no,yes,no,telephone,may,mon,2,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,Adult,0-180
36344,retired,divorced,university degree,no,no,no,cellular,jun,tue,2,999,0,nonexistent,-2.9,92.963,-40.8,1.262,5076.2,Elder,360-540
21806,admin.,single,university degree,no,no,no,cellular,aug,wed,2,999,0,nonexistent,1.4,93.444,-36.1,4.964,5228.1,Adult,180-360
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12425,entrepreneur,married,basic 6y,no,yes,yes,cellular,jul,mon,1,999,0,nonexistent,1.4,93.918,-42.7,4.960,5228.1,Adult,720-900
38689,admin.,single,high school,no,yes,no,cellular,nov,mon,1,3,1,success,-3.4,92.649,-30.1,0.715,5017.5,Adult,180-360
34610,blue-collar,single,basic 9y,no,yes,yes,cellular,may,thu,4,999,0,nonexistent,-1.8,92.893,-46.2,1.266,5099.1,Adult,180-360
37465,admin.,single,university degree,no,yes,yes,cellular,aug,thu,3,3,1,success,-2.9,92.201,-31.4,0.873,5076.2,Adult,180-360


In [87]:
X_train_OS.columns

Index(['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact',
       'month', 'day_of_week', 'campaign', 'pdays', 'previous', 'poutcome',
       'emp.var.rate', 'cons.price.idx', 'cons.conf.idx', 'euribor3m',
       'nr.employed', 'age_bin', 'duration_bin'],
      dtype='object')

In [88]:
y_train_OS

25611    0
26010    0
297      0
36344    0
21806    0
        ..
12425    1
38689    1
34610    1
37465    1
40929    1
Name: y, Length: 58476, dtype: int64

In [92]:
pipe_DT.fit(X_train_OS, y_train_OS)

Pipeline(steps=[('prep',
                 ColumnTransformer(transformers=[('numeric',
                                                  Pipeline(steps=[('scaler',
                                                                   RobustScaler())]),
                                                  ['campaign', 'pdays',
                                                   'previous', 'emp.var.rate',
                                                   'cons.price.idx',
                                                   'cons.conf.idx', 'euribor3m',
                                                   'nr.employed']),
                                                 ('categorical',
                                                  Pipeline(steps=[('encoder',
                                                                   OneHotEncoder())]),
                                                  ['job', 'marital',
                                                   'education', 'default',
        

In [93]:
conf_mat(pipe_DT, X_train_OS, X_test, y_train_OS, y_test, 'DT Over Sampling')

Classification report data TEST DT Over Sampling

               precision    recall  f1-score   support

           0       0.94      0.94      0.94      7310
           1       0.52      0.50      0.51       928

    accuracy                           0.89      8238
   macro avg       0.73      0.72      0.72      8238
weighted avg       0.89      0.89      0.89      8238

Classification report data TRAIN DT Over Sampling

               precision    recall  f1-score   support

           0       1.00      1.00      1.00     29238
           1       1.00      1.00      1.00     29238

    accuracy                           1.00     58476
   macro avg       1.00      1.00      1.00     58476
weighted avg       1.00      1.00      1.00     58476

Confusion matrix data test DT Over Sampling



,Pred1,Pred0
Akt1,465,463
Akt0,434,6876


In [95]:
pipe_KNN.fit(X_train_OS, y_train_OS)

Pipeline(steps=[('prep',
                 ColumnTransformer(transformers=[('numeric',
                                                  Pipeline(steps=[('scaler',
                                                                   RobustScaler())]),
                                                  ['campaign', 'pdays',
                                                   'previous', 'emp.var.rate',
                                                   'cons.price.idx',
                                                   'cons.conf.idx', 'euribor3m',
                                                   'nr.employed']),
                                                 ('categorical',
                                                  Pipeline(steps=[('encoder',
                                                                   OneHotEncoder())]),
                                                  ['job', 'marital',
                                                   'education', 'default',
        

In [96]:
conf_mat(pipe_KNN, X_train_OS, X_test, y_train_OS, y_test, 'KNN Over Sampling')

Classification report data TEST KNN Over Sampling

               precision    recall  f1-score   support

           0       0.96      0.88      0.92      7310
           1       0.42      0.68      0.52       928

    accuracy                           0.86      8238
   macro avg       0.69      0.78      0.72      8238
weighted avg       0.90      0.86      0.87      8238

Classification report data TRAIN KNN Over Sampling

               precision    recall  f1-score   support

           0       1.00      0.91      0.95     29238
           1       0.91      1.00      0.95     29238

    accuracy                           0.95     58476
   macro avg       0.96      0.95      0.95     58476
weighted avg       0.96      0.95      0.95     58476

Confusion matrix data test KNN Over Sampling



,Pred1,Pred0
Akt1,631,297
Akt0,877,6433


In [98]:
X_test.isna().sum()

job               0
marital           0
education         0
default           0
housing           0
loan              0
contact           0
month             0
day_of_week       0
campaign          0
pdays             0
previous          0
poutcome          0
emp.var.rate      0
cons.price.idx    0
cons.conf.idx     0
euribor3m         0
nr.employed       0
age_bin           0
duration_bin      0
dtype: int64

In [100]:
conf_mat(pipe_DT, X_train_OS, X_test, y_train_OS, y_test, 'DT Over Sampling Label Encoding')

Classification report data TEST DT Over Sampling Label Encoding

               precision    recall  f1-score   support

           0       0.94      0.94      0.94      7310
           1       0.52      0.50      0.51       928

    accuracy                           0.89      8238
   macro avg       0.73      0.72      0.72      8238
weighted avg       0.89      0.89      0.89      8238

Classification report data TRAIN DT Over Sampling Label Encoding

               precision    recall  f1-score   support

           0       1.00      1.00      1.00     29238
           1       1.00      1.00      1.00     29238

    accuracy                           1.00     58476
   macro avg       1.00      1.00      1.00     58476
weighted avg       1.00      1.00      1.00     58476

Confusion matrix data test DT Over Sampling Label Encoding



,Pred1,Pred0
Akt1,465,463
Akt0,434,6876


## PERBANDINGAN PRECISION DAN RECALL TIAP MODEL

================================================================================================

In [71]:
prec_rec (pipe_DT, X_test, y_test, 'pipe_DT')

,pipe_DT
Precison,0.52349
Recall,0.50431


In [101]:
prec_rec (pipe_KNN, X_test, y_test, 'pipe_DT')

KeyboardInterrupt: 

================================================================================================

_____________

## HYPERPARAMETER TUNING

In [104]:
skf = StratifiedKFold(n_splits=3)

https://towardsdatascience.com/fine-tuning-a-classifier-in-scikit-learn-66e048c21e65

In [73]:
scorers = {
    'precision_score': make_scorer(precision_score),
    'recall_score': make_scorer(recall_score)
}

========================

__DT__

========================

In [74]:
# pipe_DT.get_params()

In [102]:
param_DT = {
    'algo__max_depth': list(np.arange(0, 101, 5)) + [None],
    'algo__min_samples_leaf': np.arange(1, 1000, 50),
    'algo__max_features': [0, 0.2, 0.4, 0.6, 0.8, 1]
    # 'algo__class_weight': [None, {0:.4, 1:.6}, {0:.3, 1:.7}, {0:.2, 1:.8}, {0:.15, 1: .85}, {0:.1, 1:.9}]
}

In [105]:
DT_GS = GridSearchCV(pipe_DT, param_DT, cv=skf, n_jobs=-1, verbose=1, scoring='f1')

In [106]:
DT_GS.fit(X_train_OS, y_train_OS)

ValueError: 'f1_score' is not a valid scoring value. Use sorted(sklearn.metrics.SCORERS.keys()) to get valid options.

In [ ]:
param_KNN = {
    
}

In [131]:
DT_GS.best_params_

{'algo__max_depth': 30, 'algo__min_samples_leaf': 1}

In [114]:
pd.DataFrame(DT_GS.cv_results_).columns

Index(['mean_fit_time', 'std_fit_time', 'mean_score_time', 'std_score_time',
       'param_algo__max_depth', 'param_algo__min_samples_leaf', 'params',
       'split0_test_score', 'split1_test_score', 'split2_test_score',
       'mean_test_score', 'std_test_score', 'rank_test_score',
       'split0_train_score', 'split1_train_score', 'split2_train_score',
       'mean_train_score', 'std_train_score'],
      dtype='object')

______________
_UNTUK yng best estimator_
_____________

In [125]:
DT_Tune = DT_GS.best_estimator_

In [126]:
DT_Tune.fit(X_train_OS, y_train_OS)

Pipeline(steps=[('prep',
                 ColumnTransformer(transformers=[('numeric',
                                                  Pipeline(steps=[('scaler',
                                                                   RobustScaler())]),
                                                  ['campaign', 'pdays',
                                                   'previous', 'emp.var.rate',
                                                   'cons.price.idx',
                                                   'cons.conf.idx', 'euribor3m',
                                                   'nr.employed']),
                                                 ('categorical',
                                                  Pipeline(steps=[('encoder',
                                                                   OneHotEncoder())]),
                                                  ['job', 'marital',
                                                   'education', 'default',
        

In [127]:
conf_mat(DT_Tune, X_train_OS, X_test, y_train_OS, y_test, 'DT Tuning best estimator')

Classification report data TEST DT Tuning best estimator

               precision    recall  f1-score   support

           0       0.94      0.94      0.94      7310
           1       0.52      0.52      0.52       928

    accuracy                           0.89      8238
   macro avg       0.73      0.73      0.73      8238
weighted avg       0.89      0.89      0.89      8238

Classification report data TRAIN DT Tuning best estimator

               precision    recall  f1-score   support

           0       1.00      0.99      1.00     29238
           1       0.99      1.00      1.00     29238

    accuracy                           1.00     58476
   macro avg       1.00      1.00      1.00     58476
weighted avg       1.00      1.00      1.00     58476

Confusion matrix data test DT Tuning best estimator



,Pred1,Pred0
Akt1,487,441
Akt0,457,6853


______________
_{
    'algo__max_depth': 25,
    'algo__min_samples_leaf': 6,
    'algo__class_weight': None
}_
_____________

In [98]:
pipe_DT_1 = Pipeline([
    ('prep', preprocessor),
    ('algo', DecisionTreeClassifier(max_depth=25, min_samples_leaf=6, random_state=42))
])

In [99]:
pipe_DT_1.fit(X_train_OS, y_train_OS)

Pipeline(steps=[('prep',
                 ColumnTransformer(transformers=[('numeric',
                                                  Pipeline(steps=[('scaler',
                                                                   RobustScaler())]),
                                                  ['campaign', 'pdays',
                                                   'previous', 'emp.var.rate',
                                                   'cons.price.idx',
                                                   'cons.conf.idx', 'euribor3m',
                                                   'nr.employed']),
                                                 ('categorical',
                                                  Pipeline(steps=[('encoder',
                                                                   OneHotEncoder())]),
                                                  ['job', 'marital',
                                                   'education', 'default',
        

In [100]:
conf_mat(pipe_DT_1, X_train_OS, X_test, y_train_OS, y_test, 'DT Tuning 1')

Classification report data TEST DT Tuning 1

               precision    recall  f1-score   support

           0       0.96      0.90      0.93      7310
           1       0.47      0.73      0.58       928

    accuracy                           0.88      8238
   macro avg       0.72      0.82      0.75      8238
weighted avg       0.91      0.88      0.89      8238

Classification report data TRAIN DT Tuning 1

               precision    recall  f1-score   support

           0       0.98      0.93      0.95     29238
           1       0.93      0.98      0.96     29238

    accuracy                           0.96     58476
   macro avg       0.96      0.96      0.96     58476
weighted avg       0.96      0.96      0.96     58476

Confusion matrix data test DT Tuning 1



,Pred1,Pred0
Akt1,681,247
Akt0,756,6554
